# common

In [ ]:
// // test

open testing

()



## types

In [ ]:
nominal any = $"obj"
nominal disposable = $"System.IDisposable"
nominal exn = $"exn"
nominal guid = $"System.Guid"
nominal unativeint = $"unativeint"

let array x = x

inl unativeint forall t {int}. (n : t) : unativeint =
    $"unativeint !n"

()



## prototype

In [ ]:
prototype (~:>) r : forall t. t -> r
prototype append t : t -> t -> t
prototype of_string t : string -> t
prototype to_string t : t -> string

()



## null

In [ ]:
inl null forall t. () : t =
    $"null |> unbox<`t>"

()



In [ ]:
inl i64 forall t. (x : t) : i64 =
    $"int64 !x"

()



## pair

In [ ]:
type pair a b = $"(`a * `b)"

inl pair x y =
    x, y

()



In [ ]:
// // test

pair 1i32 2i32
|> _assert_eq (1, 2)

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{struct (1, 2)} / expected: %A{struct (1, 2)}"
    ()
method0()



## new_pair

In [ ]:
inl new_pair forall a b. (a : a) (b : b) : pair a b =
    $"!a, !b"

()



## from_pair

In [ ]:
inl from_pair forall a b. (pair : pair a b) : a * b =
    $"let (a, b) = !pair"
    $"a", $"b"

()



In [ ]:
// // test

new_pair "a" 1i32
|> from_pair
|> _assert_eq ("a", 1i32)

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "a"
    let v1 : (string * int32) = v0, 1
    let (a, b) = v1
    let v2 : string = a
    let v3 : int32 = b
    let v4 : bool = v2 = "a"
    let v6 : bool =
        if v4 then
            let v5 : bool = v3 = 1
            v5
        else
            false
    let v8 : bool =
        if v6 then
            true
        else
            method1(v6)
    let v9 : string = $"__expect / actual: %A{struct (v2, v3)} / expected: %A{struct (v0, 1)}"
    let v10 : bool = v8 = false
    if v10 then
        failwith<unit> v9
method0()



In [ ]:
// // test

new_pair "a" (new_pair 1i32 "b")
|> from_pair

(a, (1, b)) Item1 a Item2 (1, b) Item1 1 Item2 b

let rec method0 () : struct (string * (int32 * string)) =
    let v0 : string = "b"
    let v1 : (int32 * string) = 1, v0
    let v2 : string = "a"
    let v3 : (string * (int32 * string)) = v2, v1
    let (a, b) = v3
    let v4 : string = a
    let v5 : (int32 * string) = b
    struct (v4, v5)
method0()



## log_level_type

In [ ]:
union log_level_type =
    | Verbose
    | Debug
    | Info
    | Warning
    | Critical

instance to_string log_level_type = function
    | Verbose => "Verbose"
    | Debug => "Debug"
    | Info => "Info"
    | Warning => "Warning"
    | Critical => "Critical"

instance of_string log_level_type = function
    | "Verbose" => Verbose
    | "Debug" => Debug
    | "Info" => Info
    | "Warning" => Warning
    | "Critical" => Critical

()



## to_any

In [ ]:
inl to_any forall t. (obj : t) : any =
    $"!obj"

instance (~:>) any = to_any

()



## (/@)

In [ ]:
inl (/@) a b =
    b |> append a

()



## unbox

In [ ]:
inl unbox forall t u. (x : t) : u =
    $"!x |> unbox<`u>"

()



## (/+)

In [ ]:
inl (/+) forall t. (a : t) (b : t) : t =
    $"!a + !b"

()



## (||>)

In [ ]:
inl (||>) (arg1, arg2) fn =
    arg2 |> fn arg1

()



In [ ]:
// // test

(3i32, 2i32)
||> fun a b => a - b
|> _assert_eq 1

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{1} / expected: %A{1}"
    ()
method0()



## flip

In [ ]:
inl flip fn a b =
    fn b a

()



In [ ]:
// // test

(1i32, 2i32)
||> flip pair
|> _assert_eq (2, 1)

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{struct (2, 1)} / expected: %A{struct (2, 1)}"
    ()
method0()



## join_body

In [ ]:
inl join_body body acc x =
    if var_is x |> not
    then body acc x
    else
        inl acc = dyn acc
        join body acc x

()



In [ ]:
// // test

inl rec fold_list f s = function
    | Cons (x, x') => fold_list f (f s x) x'
    | Nil => s

()



In [ ]:
// // test

[5i32; 4; join 3; 2; 1]
|> fold_list (+) 0
|> _assert_eq 15

let rec method1 () : int32 =
    3
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : int32 = method1()
    let v1 : int32 = 9 + v0
    let v2 : int32 = v1 + 2
    let v3 : int32 = v2 + 1
    let v4 : bool = v3 = 15
    let v6 : bool =
        if v4 then
            true
        else
            method2(v4)
    let v7 : string = $"__expect / actual: %A{v3} / expected: %A{15}"
    let v8 : bool = v6 = false
    if v8 then
        failwith<unit> v7
method0()



In [ ]:
// // test

[5i32; 4; join 3; 2; 1]
|> fold_list (join_body (+)) 0
|> _assert_eq 15

let rec method1 () : int32 =
    3
and method2 (v0 : int32, v1 : int32) : int32 =
    let v2 : int32 = v1 + v0
    v2
and method3 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : int32 = method1()
    let v1 : int32 = 9
    let v2 : int32 = method2(v0, v1)
    let v3 : int32 = v2 + 2
    let v4 : int32 = v3 + 1
    let v5 : bool = v4 = 15
    let v7 : bool =
        if v5 then
            true
        else
            method3(v5)
    let v8 : string = $"__expect / actual: %A{v4} / expected: %A{15}"
    let v9 : bool = v7 = false
    if v9 then
        failwith<unit> v8
method0()



## join_body_unit

In [ ]:
inl join_body_unit body d x =
    if var_is d |> not
    then body x
    else
        inl x = dyn x
        join body x

()



In [ ]:
// // test

[5i32; 4; join 3; 2; 1]
|> fold_list (fun acc n => join_body_unit ((+) acc) n n) 0
|> _assert_eq 15

let rec method1 () : int32 =
    3
and method2 (v0 : int32) : int32 =
    let v1 : int32 = 9 + v0
    v1
and method3 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : int32 = method1()
    let v1 : int32 = method2(v0)
    let v2 : int32 = v1 + 2
    let v3 : int32 = v2 + 1
    let v4 : bool = v3 = 15
    let v6 : bool =
        if v4 then
            true
        else
            method3(v4)
    let v7 : string = $"__expect / actual: %A{v3} / expected: %A{15}"
    let v8 : bool = v6 = false
    if v8 then
        failwith<unit> v7
method0()



## run_target

In [ ]:
union target_runtime =
    | Native
    | Wasm

union target =
    | Rust : target_runtime
    | Fsharp

inl run_target forall t. (fn : target -> (() -> t)) : t =
    $"let mutable result = None"
    $"#if FABLE_COMPILER_RUST"
    $"#if \!WASM"
    fn (Rust Native) () |> fun x => $"!x"
    $"#else"
    fn (Rust Wasm) () |> fun x => $"!x"
    $"#endif"
    $"#else"
    fn Fsharp () |> fun x => $"!x"
    $"#endif"
    $"|> fun x -> result <- Some x"
    $"result |> Option.get"

()



In [ ]:
// // test

run_target function
    | Fsharp => fun () => $"1uy"
    | Rust (Native) => fun () => $"2uy"
    | Rust (Wasm) => fun () => $"3uy"
|> _assert_eq 1u8

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let mutable result = None
    #if FABLE_COMPILER_RUST
    #if !WASM
    let v0 : uint8 = 2uy
    v0
    #else
    let v1 : uint8 = 3uy
    v1
    #endif
    #else
    let v2 : uint8 = 1uy
    v2
    #endif
    |> fun x -> result <- Some x
    let v3 : uint8 = result |> Option.get
    let v4 : bool = v3 = 1uy
    let v6 : bool =
        if v4 then
            true
        else
            method1(v4)
    let v7 : string = $"__expect / actual: %A{v3} / expected: %A{1uy}"
    let v8 : bool = v6 = false
    if v8 then
        failwith<unit> v7
method0()



In [ ]:
// // test
// // rust=

run_target function
    | Fsharp => fun () => $"1uy"
    | Rust (Native) => fun () => $"2uy"
    | Rust (Wasm) => fun () => $"3uy"
|> (=) 2u8
|> _assert_eq true

.rs output:




.fsx:
let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let mutable result = None
    #if FABLE_COMPILER_RUST
    #if !WASM
    let v0 : uint8 = 2uy
    v0
    #else
    let v1 : uint8 = 3uy
    v1
    #endif
    #else
    let v2 : uint8 = 1uy
    v2
    #endif
    |> fun x -> result <- Some x
    let v3 : uint8 = result |> Option.get
    let v4 : bool = 2uy = v3
    let v6 : bool =
        if v4 then
            true
        else
            method1(v4)
    let v7 : string = $"__expect / actual: %A{v4} / expected: %A{true}"
    let v8 : bool = v6 = false
    if v8 then
        failwith<unit> v7
method0()

.rs:
#![allow(dead_code,)]
#![allow(non_camel_case_types,)]
#![allow(non_snake_case,)]
#![allow(non_upper_case_globals,)]
#![allow(unreachable_code,)]
#![allow(unused_attributes,)]
#![allow(unused_imports,)]
#![allow(unused_macros,)]
#![allow(unused_parens,)]
#![allow(unused_variables,)]
mod module_ccfa04bf {
    pub mod Spiral_eval {
        use super::*;
    

## nameof

In [ ]:
inl nameof x : string =
    $"nameof !x"

()



## get_environment_variable

In [ ]:
inl get_environment_variable (var : string) : string =
    $"System.Environment.GetEnvironmentVariable !var"

()



## new_raw_guid

In [ ]:
inl new_raw_guid () : guid =
    $"`guid.NewGuid ()"

()

